In [1]:
import pandas as pd

listings = pd.read_csv('./data/listings.csv')
reviews = pd.read_csv('./data/reviews.csv')

In [14]:
reviews.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [2]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

## Reviewer

In [15]:
reviewer = pd.DataFrame()
reviewer['reviewer_id'] = reviews['reviewer_id']
reviewer['reviewer_name'] = reviews['reviewer_name']

In [71]:
reviewer_final = reviewer.drop_duplicates(subset=['reviewer_id'])
reviewer_final = reviewer_final.reset_index(drop=True)

In [72]:
reviewer_final.to_csv('./data_sql/reviewer.csv', index=False, sep=';')

## Date

In [17]:
from datetime import datetime

def get_day(date): 
    data = datetime.strptime(date,'%Y-%m-%d')
    return data.day

def get_month(date):
    data = datetime.strptime(date,'%Y-%m-%d')
    return data.month

def get_year(date):
    data = datetime.strptime(date,'%Y-%m-%d')
    return data.year


In [38]:
date = pd.DataFrame() 
date['date'] = reviews['date']

In [42]:
#drop duplicates
date = date.drop_duplicates()
date = date.reset_index(drop=True)

In [ ]:
#foreign key for day

size_date = len(date.index)
date['day_id'] = [i for i in range(size_date)]

for i in date['date'].index:
    date['day_id'][i] = get_day(date['date'][i])

In [ ]:
#foreign key for month

size_date = len(date.index)
date['month_id'] = [i for i in range(size_date)]
for i in date['date'].index:
    date['month_id'][i] = get_month(date['date'][i])

In [52]:
# Table day
day = pd.DataFrame()
day['day_number'] = [i for i in range(1,32)]

In [53]:
# Table month
month = pd.DataFrame()
month['month_number'] = [i for i in range(1,13)]
month['month_name'] = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [54]:
max = date['date'].max()
min = date['date'].min()

print(max, min)

2023-09-11 2010-07-22


In [55]:
# Table year -> min_year(date) - max_year(date)
year = pd.DataFrame()
year['year_number'] = [i for i in range(2010,2024)]

In [ ]:
# Foreign key of year_id

size_date = len(date.index)
date['year_id'] = [i for i in range(size_date)]
for i in date['date'].index:
    year_number = get_year(date['date'][i])
    row_num = year[year['year_number']==year_number].index
    date['year_id'][i] = row_num[0]+1

In [62]:
day.to_csv('./data_sql/day.csv', sep=';', index=False)
month.to_csv('./data_sql/month.csv', sep=';', index=False)
year.to_csv('./data_sql/year.csv', sep=';', index=False)
date.to_csv('./data_sql/date.csv', sep=';', index=False)

In [69]:
date = date.reset_index(drop=True)

## Availability

In [248]:
# Availability 
selected_columns = ['availability_30', 'availability_60', 'availability_90', 'availability_365']

availability = listings[selected_columns].copy()

In [249]:
availability = availability.drop_duplicates().reset_index(drop=True)

In [27]:
availability.to_csv('./data_sql/availabilty.csv', index=False, sep=';')

## Location

In [32]:
location = pd.DataFrame()
location['latitude'] = listings['latitude']
location['longitude'] = listings['longitude']

In [36]:
location = location.drop_duplicates().reset_index(drop=True)

In [38]:
country = pd.DataFrame({'country': ['Portugal']})

In [41]:
size_location = len(location.index)
location['country_id'] = [1 for i in range(size_location)]

In [52]:
city = pd.DataFrame({'city': ['Porto']})

In [53]:
size_location = len(location.index)
location['city_id'] = [1 for i in range(size_location)]

In [47]:
country_ = pd.DataFrame({'country_id': 0, 'country': ['Portugal'], 'country_code': ['PT']})

# Create a new dataframe that came from listings that take the city

state = pd.DataFrame()


state["state"] = listings["neighbourhood_group_cleansed"].drop_duplicates().reset_index(drop=True)
state["state_id"] = state.index
state["country_id"] = 0

# Create a new dataframe that came from listings that take the neighbourhood

neighbourhood = pd.DataFrame()

# Extract the 'neighbourhood_cleansed' column and drop duplicates
neighbourhood['combined_neighbourhood'] = listings['neighbourhood_cleansed'].drop_duplicates().reset_index(drop=True)

# Assuming 'city_column' is the column in 'listings' containing city information
neighbourhood = pd.merge(neighbourhood, listings[['neighbourhood_cleansed', 'neighbourhood_group_cleansed']], 
                         left_on='combined_neighbourhood', right_on='neighbourhood_cleansed', how='left')

neighbourhood.rename(columns={'neighbourhood_group_cleansed': 'state_column'}, inplace=True)
neighbourhood.drop(['neighbourhood_cleansed'], axis=1, inplace=True)
neighbourhood.drop_duplicates().reset_index(drop=True, inplace=True)


# Split the combined neighborhood string into individual neighborhoods using both commas and " e "
neighbourhood['neighbourhood'] = neighbourhood['combined_neighbourhood'].str.split(',| e ')

# Explode the DataFrame, creating a new row for each neighborhood
neighbourhood = neighbourhood.explode('neighbourhood')

# Drop unnecessary columns and rename columns
neighbourhood.rename(columns={'state_column': 'state'}, inplace=True)
neighbourhood.drop_duplicates().reset_index(drop=True, inplace=True)
neighbourhood = pd.merge(neighbourhood, state, on="state", how='left')
neighbourhood.drop(['combined_neighbourhood', 'state', 'country_id'], axis=1, inplace=True)

## Listing Score

In [173]:
selected_columns = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

listing_score = listings[selected_columns].copy()

In [174]:
listing_score=listing_score.drop_duplicates().reset_index(drop=True)

In [175]:
listing_score.to_csv('./data_sql/listing_score.csv', index=False, sep=';')

## Host

host_listing_count é para por, 
description == host_description?

In [43]:
selected_columns = ['host_id', 'host_name', 'host_url', 'host_total_listings_count', 'host_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'host_is_superhost', 'description', 'reviews_per_month']

host = listings[selected_columns].copy()

In [44]:
host = host.drop_duplicates().reset_index(drop=True)

In [14]:
selected_columns = ['host_response_time', 'host_response_rate', 'host_acceptance_rate']

host_response = listings[selected_columns].copy()

In [20]:
host_response = host_response.drop_duplicates().reset_index(drop=True)

In [75]:
def get_host_response_id(host_id):
    listing_host = listings[listings['host_id']==host_id]
    list_host_response_time = listing_host['host_response_time'].iloc[0]
    list_host_response_rate = listing_host['host_response_rate'].iloc[0]
    list_host_acceptance_rate = listing_host['host_acceptance_rate'].iloc[0]
    row = host_response[(host_response['host_response_time']==list_host_response_time)
                         & (host_response['host_response_rate']==list_host_response_rate)
                         & (host_response['host_acceptance_rate']==list_host_acceptance_rate)]
    return row.index[0] + 1 

In [ ]:
# host_response_id -> run
size_host = len(host.index)
host['host_response_id'][i] = [i for i in range(size_host)]

for i in range(size_host):
    host_id = host['host_id'][i]
    host['host_response_id'][i] = get_host_response_id(host_id)

In [112]:
host_verification = pd.DataFrame()
host_verification['host_verifications'] = listings['host_verifications']

In [113]:
host_verification = host_verification.drop_duplicates().reset_index(drop=True)

In [121]:
def get_host_verifications_id(host_id):
    listing_host = listings[listings['host_id']==host_id]
    list_host_verifications = listing_host['host_verifications'].iloc[0]
    
    row = host_verification[host_verification['host_verifications']==list_host_verifications]
    return row.index[0] + 1 

In [ ]:
# host_verifications_id -> run
size_host = len(host.index)
host['host_verifications_id'][i] = [i for i in range(size_host)]

for i in range(size_host):
    host_id = host['host_id'][i]
    host['host_verifications_id'][i] = get_host_verifications_id(host_id)

## Property

square_feet parece que não existe

In [152]:
selected_columns = ['accommodates', 'bedrooms', 'beds', 'amenities']

property = listings[selected_columns].copy()

In [171]:
property = property.drop_duplicates().reset_index(drop=True)

In [154]:
room_type = pd.DataFrame()
room_type['room_type'] = listings['room_type']

In [156]:
room_type = room_type.drop_duplicates().reset_index(drop=True)

In [158]:
property_type = pd.DataFrame()
property_type['property_type'] = listings['property_type']

In [160]:
property_type = property_type.drop_duplicates().reset_index(drop=True)

In [166]:
bathroom_type = pd.DataFrame()
bathroom_type['bathrooms_text'] = listings['bathrooms_text']

In [167]:
bathroom_type = bathroom_type.drop_duplicates().reset_index(drop=True)

# Review

In [183]:
review = pd.DataFrame()
review['reviewer_id'] = reviews['reviewer_id']
review['review'] = reviews['comments']


In [82]:
def get_date_id(date_str):
    return date[date['date']==date_str].index[0]+1

In [86]:
# data_id
size_review = len(review.index)
review['date_id'] = [i for i in range(size_review)]

for i in range(size_review):
    review['date_id'][i] = get_date_id(reviews['date'][i])

C:\Users\filip\AppData\Local\Temp\ipykernel_15204\753352024.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review['date_id'][i] = get_date_id(reviews['date'][i])


In [214]:
def get_host_id(listings_id):
    l = listings[listings['id']==listings_id]
    return l['host_id'].tolist()[0]

In [212]:
# host_id -> review['listing_id'] = listings['id]->host_id

size_review = len(review.index)
review['host_id'] = [i for i in range(size_review)]

for i in range(size_review):
    review['host_id'][i] = get_host_id(reviews['listing_id'][i])

# Listing

In [244]:
#listing -> listing_id = id, host_id = host_id, price -> price

selected_columns = ['id', 'host_id', 'price']
listing = listings[selected_columns].copy()
listing.rename(columns={'id':'listing_id'}, inplace=True)

In [ ]:
# availability_id -> buscar availabilities == def get_response_id

In [ ]:
#listing_score_id -> buscar scores == def get_response_id